In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import pickle

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 500)

In [2]:
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10, duplicates = "drop")
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events'] = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100
    
    #Formatting
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    #Display KS
    print("KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [3]:
def woe_func(df, x, y):
    
    col_list = ['Variable', 'Band', 'total', 'bad', 'good', 'bad%', 'good%', 'woe', 'IV']
    df_woe_f = pd.DataFrame(columns = col_list)
    
    for i in x:
        df[str(i)+'_qband'] = pd.qcut(df[i], q = 8, duplicates = 'drop')
        df[str(i)+'_qband'] = df[str(i)+'_qband'].cat.add_categories('NULL')
        df[str(i)+'_qband'].fillna('NULL',inplace=True)

        df_grp = df.groupby([str(i)+'_qband']).agg(total = (y,'count'), bad = (y,'sum')).reset_index()
        df_grp['good'] = df_grp['total']-df_grp['bad']
        df_grp['bad%'] = (df_grp['bad'])/df_grp['bad'].sum()
        df_grp['good%'] = (df_grp['good'])/df_grp['good'].sum()
        df_grp['woe'] = np.log(df_grp['good%']/df_grp['bad%'])
        df_grp['IV'] = (df_grp['good%']-df_grp['bad%'])*df_grp['woe']

        df_grp.rename(columns={str(i)+'_qband':'Band'},inplace=True)
        df_grp.insert(0,'Variable','')
        df_grp['Variable'] = i

        df_woe_f = pd.concat([df_woe_f,df_grp], axis=0).reset_index(drop=True)
    
    return df_woe_f

In [4]:
def woe_func_cat(df, x, y):
    
    col_list = ['Variable', 'Band', 'total', 'bad', 'good', 'bad%', 'good%', 'woe', 'IV']
    df_woe_f = pd.DataFrame(columns = col_list)
    
    for i in x:
        
        df_grp = df.groupby([i]).agg(total = (y,'count'), bad = (y,'sum')).reset_index()
        df_grp['good'] = df_grp['total']-df_grp['bad']
        df_grp['bad%'] = (df_grp['bad'])/df_grp['bad'].sum()
        df_grp['good%'] = (df_grp['good'])/df_grp['good'].sum()
        df_grp['woe'] = np.log(df_grp['good%']/df_grp['bad%'])
        df_grp['IV'] = (df_grp['good%']-df_grp['bad%'])*df_grp['woe']

        df_grp.rename(columns={i:'Band'},inplace=True)
        df_grp.insert(0,'Variable','')
        df_grp['Variable'] = i

        df_woe_f = pd.concat([df_woe_f,df_grp], axis=0).reset_index(drop=True)
    
    return df_woe_f

In [5]:
# Preprocessing of dataset

df = pd.read_csv(r'etb_cap_final_train.csv',encoding= 'utf-8')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head(5)

,apprefno,activation_vintage,cust_id__c,bureau_live_pl_cnt,bureau_closed_PL_cnt,bureau_Live_PL_amt,bureau_closed_PL_amt,bureau_live_hl_cnt,bureau_closed_HL_cnt,bureau_Live_HL_amt,bureau_closed_HL_amt,bureau_unsec_live_cnt,bureau_unsec_closed_cnt,bureau_sec_live_cnt,bureau_sec_closed_cnt,bureau_first_live_cc_dt,bureau_live_cc_cnt,bureau_live_cc_sanc_amt,bureau_live_cc_bal_amt,bureau_min_closed_cc_dt,bureau_total_sanction_amt,bureau_total_loan_cnt,bureau_total_live_loan_cnt,bureau_total_secured_sanction_amt,bureau_total_unsecured_sanction_amt,bureau_closed_cc_cnt,bureau_TOT_SEC_LOAN,bureau_TOT_UNSEC_LOAN,bureau_TOT_CC_LOAN,TOT_LIVE_UNSEC_LOAN1,TOT_UNSEC_LOAN_new,TOT_LIVE_UNSEC_LOAN_new,max_live_sanc_amt_new,max_cc_limit_new,bureau_min_vintage,bureau_first_live_cc_vintage,bureau_first_closed_cc_vintage,bureau_first_Loan_vintage,bureau_Last_Loan_vintage,utilization_percent,utilization_percent_new,onus_plcs_all_amt_Fin,onus_Live_plcs_amt_Fin,onus_Sum_live_loan_EMI_cnt,onus_B2B_live_loan_cnt,onus_B2B_loan_cnt_ever,onus_plcs_live_loan_cnt,onus_plcs_loan_cnt_ever,onus_salpl_live_loan_cnt,onus_salpl_loan_cnt_ever,onus_salpl_live_amt_fin,onus_salpl_all_amt_fin,onus_avg_live_loan_EMI_amt,onus_Sum_live_loan_EMI_amt,onus_Overall_max_loan_EMI_amt,onus_Overall_avg_loan_EMI_amt,onus_avg_live_loan_DISB_amt,onus_Sum_live_loan_DISB_amt,onus_avg_loan_DISB_amt,onus_Sum_loan_DISB_amt,onus_avg_live_loan_FIN_amt,onus_Sum_live_loan_FIN_amt,onus_avg_loan_FIN_amt,onus_Sum_loan_FIN_amt,onus_avg_plcs_live_loan_DISB_amt,onus_avg_plcs_all_loan_DISB_amt,onus_sum_plcs_live_loan_DISB_amt,onus_sum_plcs_all_loan_DISB_amt,onus_avg_SALPL_live_loan_DISB_amt,onus_avg_SALPL_all_loan_DISB_amt,onus_sum_SALPL_live_loan_DISB_amt,onus_sum_SALPL_all_loan_DISB_amt,onus_avg_B2B_live_loan_DISB_amt,onus_avg_B2B_all_loan_DISB_amt,onus_sum_B2B_live_loan_DISB_amt,onus_sum_B2B_all_loan_DISB_amt,onus_ECOM_live_loan_cnt,onus_ECOM_loan_cnt_ever,onus_avg_ECOM_live_loan_DISB_amt,onus_avg_ECOM_all_loan_DISB_amt,onus_sum_ECOM_live_loan_DISB_amt,onus_sum_ECOM_all_loan_DISB_amt,onus_latest_disb_loan_vintage,onus_first_disb_loan_vintage,onus_latest_closed_cc_vintage,onus_latest_SALPL_loan_vintage,onus_first_SALPL_loan_vintage,onus_latest_PLCS_loan_vintage,onus_first_PLCS_loan_vintage,onus_latest_B2B_loan_vintage,onus_first_B2B_loan_vintage,onus_latest_ECOM_loan_vintage,onus_first_ECOM_loan_vintage,activation_flag,CIR_ACTIVATION_VINTAGE,CIR_live_pl_cnt,CIR_closed_PL_cnt,CIR_Live_PL_amt,CIR_closed_PL_amt,CIR_live_hl_cnt,CIR_closed_HL_cnt,CIR_Live_HL_amt,CIR_closed_HL_amt,CIR_TOT_LOAN_CNT,CIR_TOT_LOAN_SANC_AMT,CIR_TOT_LIVE_LOAN_CNT,CIR_TOT_LIVE_LOAN_SANC_AMT,CIR_TOT_SEC_LOAN,CIR_total_secured_sanction_amt,CIR_TOT_LIVE_SEC_LOAN,CIR_TOT_LIVE_SEC_SANC_AMT,CIR_TOT_UNSEC_LOAN,CIR_total_unsecured_sanction_amt,CIR_TOT_LIVE_UNSEC_LOAN,CIR_TOT_LIVE_UNSEC_SANC_AMT,CIR_TOT_CC_LOAN,CIR_TOT_CC_SANC_AMT,CIR_live_cc_cnt,CIR_live_cc_bal_amt,CIR_live_CC_sanc_amt,CIR_EVER_CC_FLAG1,CIR_CC_ACTIVE_COUNT,CIR_CC_NC_FLAG1,CIR_THICK_THIN_FLAG1,CIR_OLDEST_LIVE_CC_VINTAGE,CIR_CC_UTIL_PERC,CIR_SC_MC,CIR_BUREAU_VINTAGE,CIR_CC_UTIL_PERC_new,round_utilization_percent,round_utilization_percent_new,B2B_FLAG,App_installation_flag1,age_nov21,MARITAL_STATUS1,EMP_TYPE_FINAL_NOV21,ESTIMATED_INCOME_NOV21,EMI_MAX_NOV21,City_Tier_NOV21,ANNUAL_INCOME,GROSS_INCOME,EMI_CARD_LIMIT,YEARS_CURRENT_JOB,CUSTOMER_COMPANY_CATEGORY_PL,CUST_EMI_CARD_BLOCK_STATUS,CUST_IMPUTED_INCOME_SEGMENT1,BFL_TRANSACTION_AFFINITY,CUST_IMPUTED_INCOME_SEGMENT_ROLLUP,AC_DAIKIN_BOUGHT,BRAND_APPLE_BOUGHT,CUST_RESIDENCE_AREA_AFFLUENCE,BEH_DAYS_GAP_BETWEEN_2_PURCHASE,UCIC_BAGIC_EVER_BOUGHT,UCIC_BALIC_EVER_BOUGHT,CUST_BANK_CLASSIFICATION,BEH_BFL_LOYALTY_ATTRITION,BEH_PURCHASE_BEHAVIOR_ON_EMI_CARD,BEH_SALES_FINANCE_PURCHASE_BEHAVIOR,CUST_CITY_TIER,BEH_BFL_CLOSED_TO_TOTAL_LOAN_RATIO,ECOM_COUNT_EVER_BOUGHT,MOBILE_COUNT_EVER_BOUGHT,REFRIGERATOR_COUNT_EVER_BOUGHT,REMI_COUNT_EVER_BOUGHT,LED_COUNT_EVER_BOUGHT,WM_COUNT_EVER_BOUGHT,DEALER_LOYALTY_FLAG,BEH_DIGITAL_SAVVY,ECOM_PRO

In [6]:
def encode_mobile(x):
    if x in ('VIVO'):
        return 'VIVO'
    elif x in ('OPPO'):
        return 'OPPO'
    elif x in ('SAMSUNG'):
        return 'SAMSUNG'
    elif x in ('MI'):
        return 'MI'
    elif x in ('APPLE'):
        return 'APPLE'
    elif x in ('ONEPLUS'):
        return 'ONEPLUS'
    elif x in ('OTHERS','REALME','NOKIA','SONY','LG','GIONEE','MOTO','HTC',
               'LENEVO','HUAWEI','ASUS','MICROMAX','HONOR','LAVA','BLACKBERRY','PANASONIC','INTEX','LYF'):
        return 'OTHERS'
    elif x in ('NO MOBILE'):
        return 'NO MOBILE'
    else:
        pass

df['MOBILE_BRAND_NAME'].fillna('NO MOBILE',inplace=True)
df['MOBILE_BRAND_NAME'] = df['MOBILE_BRAND_NAME'].map(lambda x: encode_mobile(x))

In [7]:
def encode_emp(x):
    if x in ('SAL'):
        return 'SAL'
    elif x in ('Self Employed Non - Professional'):
        return 'Self Employed Non - Professional'
    elif x in ('Self Employed – Professional','SEMP'):
        return 'SEMP'
    elif x in ('Farmer','Housewife','OTHER','Pensioner','Student','Trader'):
        return 'Others'
    else:
        pass

df['Employment_Business__c_NEW1'].fillna('Others', inplace = True)
df['Employment_Business__c_NEW1'] = df['Employment_Business__c_NEW1'].map(lambda x: encode_emp(x))

In [8]:
def encode_tag(x):
    if x in ('G1','G2','G3'):
        return 'GROWTH'
    elif x in ('P1','P2','P3'):
        return 'PRIME'
    elif x in ('OGL'):
        return 'OGL'
    elif x in ('EMERGING'):
        return 'EMERGING'
    else:
        pass

df['TAG'].fillna('OGL', inplace = True)
df['TAG'] = df['TAG'].map(lambda x: encode_tag(x))

In [9]:
df_cc = df
df_fill  = df_cc.isna().sum().to_frame().reset_index()
df_fill['base_count'] = df_cc.shape[0]
df_fill['data_fill'] = df_cc.shape[0]-df_fill[0]
df_fill['fill_rate'] = 100-((df_fill[0]*100)/df_cc.shape[0])
df_zero = (df_cc == 0).astype(int).sum(axis=0).to_frame().reset_index()
df_fill = pd.merge(df_fill, df_zero, how='left', on='index')

In [10]:
data_univar = df_cc.describe(percentiles=[0.01,0.02,0.03,0.05,0.1,.25,0.5,0.75,0.90,0.95,0.97,0.98,0.99]).reset_index()
num_col = df_cc.select_dtypes(include=['int64','float64']).columns.tolist()
num_col = list(set(num_col)-set(['activation_flag']))
df_cc[num_col] = df_cc[num_col].apply(lambda x: x.clip(*x.quantile([0.05, 0.99])))
data_univar_C = df_cc.describe(percentiles=[0.01,0.02,0.03,0.05,0.1,.25,0.5,0.75,0.90,0.95,0.97,0.98,0.99]).reset_index()

## Drop unwanted Variables

In [11]:
# On the basis of fill rate, CIR columns removed as discussed
data1= df_cc.drop(['MOBILE_BRAND_NAME','apprefno','cust_id__c','bureau_closed_HL_cnt','bureau_first_live_cc_dt','bureau_min_closed_cc_dt','bureau_TOT_SEC_LOAN',
                   'bureau_TOT_UNSEC_LOAN','TOT_LIVE_UNSEC_LOAN_new','onus_plcs_all_amt_Fin','onus_Live_plcs_amt_Fin','onus_plcs_live_loan_cnt','onus_plcs_loan_cnt_ever',
                   'onus_salpl_live_loan_cnt','onus_salpl_loan_cnt_ever','onus_salpl_live_amt_fin','onus_salpl_all_amt_fin','onus_avg_plcs_live_loan_DISB_amt',
                   'onus_avg_plcs_all_loan_DISB_amt','onus_sum_plcs_live_loan_DISB_amt','onus_sum_plcs_all_loan_DISB_amt','onus_avg_SALPL_live_loan_DISB_amt',
                   'onus_avg_SALPL_all_loan_DISB_amt','onus_sum_SALPL_live_loan_DISB_amt','onus_sum_SALPL_all_loan_DISB_amt','onus_latest_SALPL_loan_vintage',
                   'onus_first_SALPL_loan_vintage','onus_latest_PLCS_loan_vintage','onus_first_PLCS_loan_vintage','CIR_ACTIVATION_VINTAGE','CIR_live_hl_cnt',
                   'CIR_closed_HL_cnt','CIR_EVER_CC_FLAG1','CIR_CC_NC_FLAG1','CIR_THICK_THIN_FLAG1','ANNUAL_INCOME','GROSS_INCOME','CUSTOMER_COMPANY_CATEGORY_PL',
                   'CUST_EMI_CARD_BLOCK_STATUS','AC_DAIKIN_BOUGHT','UCIC_BALIC_EVER_BOUGHT','CARDED_NON_CARDED','TVR','activation_vintage','Employment_Business__c'],axis=1)

## Categorical Missing Value Treatment

In [12]:
# Mode imputation (cat columns)
mode_imp_var = ['CIR_SC_MC','EMP_TYPE_FINAL_NOV21','City_Tier_NOV21','CUST_IMPUTED_INCOME_SEGMENT1','BFL_TRANSACTION_AFFINITY','CUST_IMPUTED_INCOME_SEGMENT_ROLLUP',
                'BRAND_APPLE_BOUGHT','CUST_RESIDENCE_AREA_AFFLUENCE','BEH_DAYS_GAP_BETWEEN_2_PURCHASE','UCIC_BAGIC_EVER_BOUGHT','CUST_BANK_CLASSIFICATION',
                'BEH_BFL_LOYALTY_ATTRITION','BEH_PURCHASE_BEHAVIOR_ON_EMI_CARD','BEH_SALES_FINANCE_PURCHASE_BEHAVIOR','CUST_CITY_TIER','BEH_BFL_CLOSED_TO_TOTAL_LOAN_RATIO',
                'ECOM_COUNT_EVER_BOUGHT','MOBILE_COUNT_EVER_BOUGHT','REFRIGERATOR_COUNT_EVER_BOUGHT','REMI_COUNT_EVER_BOUGHT','LED_COUNT_EVER_BOUGHT','WM_COUNT_EVER_BOUGHT',
                'DEALER_LOYALTY_FLAG','BEH_DIGITAL_SAVVY','ECOM_PROPENSITY_TO_BUY','ECOM_EVER_BOUGHT','BEH_EMI_RATIO','ONUS_EMI_CARD_LAST_SALES_FINANCE_PRODUCT_BOUGHT',
                'ONUS_LAST_SALES_FINANCE_PRODUCT_BOUGHT','ONUS_OFFUS_LAST_LOAN_PRODUCT_BOUGHT','BEH_MEDIUM_CLASS_BRAND_BUYER','BEH_MEDIUM_CLASS_PRICE_BUYER',
                'MOBILE_APPLE_BOUGHT','MOBILE_LAUNCH_BEHAVIOR','BEH_PREMIUM_CLASS_BRAND_BUYER','BEH_PREMIUM_CLASS_PRICE_BUYER','BRAND_SAMSUNG_BOUGHT',
                'BRAND_SAMSUNG_MOB_BOUGHT','BRAND_SONY_BOUGHT','BEH_SUB_PREMIUM_CLASS_BRAND_BUYER','BEH_SUB_PREMIUM_CLASS_PRICE_BUYER','EMI_CARD_ARU_SEGMENT',
                'BEH_TIME_SINCE_LAST_LOAN_EMI_CARD','CUST_WALLET_DIGITIZE_FLAG','Employment_Business__c_NEW1','MOBILE_AFFORDABLE_SCHEME_SEEKER',
                'MOBILE_SAMSUNG_BOUGHT','MOBILE_VIVO_BOUGHT','ECOM_FLIPKART_MOBILE_BOUGHT']

for i in mode_imp_var :
    data1[i].fillna(data1[i].mode()[0],inplace=True)

# Mean imputation columns
mean_imp_var = ['bureau_live_cc_bal_amt','bureau_total_sanction_amt','bureau_total_unsecured_sanction_amt','max_live_sanc_amt_new',
                'max_cc_limit_new','bureau_min_vintage','bureau_first_live_cc_vintage','bureau_first_Loan_vintage','bureau_Last_Loan_vintage','onus_Overall_max_loan_EMI_amt',
                'onus_Overall_avg_loan_EMI_amt','onus_avg_loan_DISB_amt','onus_Sum_loan_DISB_amt','onus_avg_loan_FIN_amt','onus_Sum_loan_FIN_amt','onus_avg_B2B_all_loan_DISB_amt',
                'onus_sum_B2B_all_loan_DISB_amt','onus_latest_disb_loan_vintage','onus_first_disb_loan_vintage','onus_latest_closed_cc_vintage','onus_latest_B2B_loan_vintage',
                'onus_first_B2B_loan_vintage','CIR_TOT_LOAN_SANC_AMT','CIR_TOT_LIVE_LOAN_SANC_AMT','CIR_total_unsecured_sanction_amt','CIR_TOT_CC_SANC_AMT',
                'CIR_live_cc_bal_amt','CIR_live_CC_sanc_amt','CIR_OLDEST_LIVE_CC_VINTAGE','age_nov21','EMI_CARD_LIMIT','BEH_EMI_CARD_VINTAGE',
                'Card_Limit__c','OFFER_VINTAGE','MAX_LIMIT','BALANCE']

for i in mean_imp_var :
    data1[i].fillna(data1[i].mean(),inplace=True)
    
# -99999 imputation columns
imp_99_var = ['bureau_Live_PL_amt','bureau_closed_PL_amt','bureau_Live_HL_amt','bureau_closed_HL_amt','bureau_total_secured_sanction_amt',
              'bureau_first_closed_cc_vintage','onus_Sum_live_loan_EMI_cnt','onus_B2B_live_loan_cnt','onus_avg_live_loan_EMI_amt',
              'onus_Sum_live_loan_EMI_amt','onus_avg_live_loan_DISB_amt','onus_Sum_live_loan_DISB_amt','onus_avg_live_loan_FIN_amt',
              'onus_Sum_live_loan_FIN_amt','onus_avg_B2B_live_loan_DISB_amt','onus_sum_B2B_live_loan_DISB_amt','onus_ECOM_live_loan_cnt','onus_ECOM_loan_cnt_ever',
              'onus_avg_ECOM_live_loan_DISB_amt','onus_avg_ECOM_all_loan_DISB_amt','onus_sum_ECOM_live_loan_DISB_amt','onus_sum_ECOM_all_loan_DISB_amt',
              'onus_latest_ECOM_loan_vintage','onus_first_ECOM_loan_vintage','CIR_live_pl_cnt','CIR_closed_PL_cnt','CIR_Live_PL_amt','CIR_closed_PL_amt',
              'CIR_Live_HL_amt','CIR_closed_HL_amt','CIR_TOT_SEC_LOAN','CIR_total_secured_sanction_amt','CIR_TOT_LIVE_SEC_LOAN','CIR_TOT_LIVE_SEC_SANC_AMT',
              'ESTIMATED_INCOME_NOV21','YEARS_CURRENT_JOB','exp_login_12m','exp_login_24m','login_recency_portal','num_ECF_REMI','num_ECF_REMI_1yr',
              'No_email_click_last_6M','No_email_click_last_12M','email_click_recency','No_email_open_last_6M','No_email_open_last_12M','email_open_recency',
              'CCPL_enq_L12m_new','unsec_enq_L12m_new','sec_enq_L12m_new','TOT_enq_L12m_new','max_CC_SANCTIONAMOUNT','UTILIZED_LIMIT']

for i in imp_99_var :
    data1[i].fillna(-99999,inplace=True)
    
# zero imputation columns
zero_imp_var = ['bureau_live_pl_cnt','bureau_closed_PL_cnt','bureau_live_hl_cnt','bureau_unsec_live_cnt','bureau_unsec_closed_cnt','bureau_sec_live_cnt',
                'bureau_sec_closed_cnt','bureau_live_cc_cnt','bureau_live_cc_sanc_amt','bureau_total_loan_cnt','bureau_total_live_loan_cnt','bureau_closed_cc_cnt',
                'bureau_TOT_CC_LOAN','TOT_LIVE_UNSEC_LOAN1','TOT_UNSEC_LOAN_new','utilization_percent','utilization_percent_new','onus_B2B_loan_cnt_ever',
                'CIR_TOT_LOAN_CNT','CIR_TOT_LIVE_LOAN_CNT','CIR_TOT_UNSEC_LOAN','CIR_TOT_LIVE_UNSEC_LOAN','CIR_TOT_LIVE_UNSEC_SANC_AMT','CIR_TOT_CC_LOAN',
                'CIR_live_cc_cnt','CIR_CC_ACTIVE_COUNT','CIR_CC_UTIL_PERC','CIR_BUREAU_VINTAGE','CIR_CC_UTIL_PERC_new','round_utilization_percent',
                'round_utilization_percent_new','EMI_MAX_NOV21']

for i in zero_imp_var :
    data1[i].fillna(0,inplace=True)

# change data types of these columns
data1['bureau_live_pl_cnt'] = pd.to_numeric(data1['bureau_live_pl_cnt'], errors='coerce')
data1['onus_ECOM_live_loan_cnt'] = pd.to_numeric(data1['onus_ECOM_live_loan_cnt'], errors='coerce')
data1['bureau_live_hl_cnt'] = pd.to_numeric(data1['bureau_live_hl_cnt'], errors='coerce')

In [13]:
data1.drop(['activation_flag_3MOB', 'activation_flag'], axis = 1, inplace = True)

## WOE Analysis

In [14]:
num_col = data1.select_dtypes(include=['int64','float64']).columns.tolist()
data_woe_temp = data1[num_col]

int_cat = data_woe_temp.nunique().to_frame().reset_index()[
    data_woe_temp.nunique().to_frame().reset_index()[0]<=5]['index'].tolist()
data_woe_temp1 = data_woe_temp[list(set(num_col)-set(int_cat))+['activation_flag_1MOB']]

y = 'activation_flag_1MOB'
x = list(set(list(set(num_col)-set(int_cat))))

data_woe_ff = woe_func(data_woe_temp1,x,y)

data_grpby_cont = data_woe_ff.groupby(['Variable']).agg(IV = ('IV','sum')).reset_index()

data_grpby_cont['IV_seg'] = ['Suspicious' if data_grpby_cont['IV'][i]>0.5
                                       else('High' if data_grpby_cont['IV'][i]<0.5 and data_grpby_cont['IV'][i]>=0.3 
                                          else('Medium' if data_grpby_cont['IV'][i]<0.3 and data_grpby_cont['IV'][i]>=0.1 
                                              else('Low' if data_grpby_cont['IV'][i]<0.1 and data_grpby_cont['IV'][i]>0.05 
                                                  else 'Not useful'))) for i in range(data_grpby_cont.shape[0])]

In [21]:
woe_df = pd.concat([data_woe_ff, data_woe_cat_f], axis = 0)


,Variable,Band,total,bad,good,bad%,good%,woe,IV
0,TOT_LIVE_UNSEC_LOAN1,"(-0.001, 1.0]",59265,15000,44265,0.384823,0.542045,0.342566,0.053859
1,TOT_LIVE_UNSEC_LOAN1,"(1.0, 2.0]",24716,8334,16382,0.213807,0.200605,-0.063738,0.000842
2,TOT_LIVE_UNSEC_LOAN1,"(2.0, 3.0]",15943,5988,9955,0.153621,0.121903,-0.231261,0.007335
3,TOT_LIVE_UNSEC_LOAN1,"(3.0, 4.0]",9235,3879,5356,0.099515,0.065587,-0.416938,0.014146
4,TOT_LIVE_UNSEC_LOAN1,"(4.0, 8.0]",11483,5778,5705,0.148234,0.069860,-0.752293,0.058960
...,...,...,...,...,...,...,...,...,...
305,MOBILE_APPLE_BOUGHT,NO_LOANS,117050,37792,79258,0.969548,0.970550,0.001033,0.000001
306,TAG,EMERGING,3063,1552,1511,0.039816,0.018503,-0.766351,0.016334
307,TAG,GROWTH,39809,15073,24736,0.386695,0.302903,-0.244223,0.020464
308,TAG,OGL,46,14,32,0.000359,0.000392,0.087101,0.000003


### Categorical Vars

In [22]:
num_col_cat = data1.select_dtypes(include=['object']).columns.tolist()+['activation_flag_1MOB']
data_woe_temp_cat = data1[list(set(num_col_cat+int_cat))]

y = 'activation_flag_1MOB'
x = list(set(num_col_cat+int_cat)-set([y]))

data_woe_cat_f = woe_func_cat(data_woe_temp_cat,x,y)

data_grpby_cat = data_woe_cat_f.groupby(['Variable']).agg(IV = ('IV','sum')).reset_index()

data_grpby_cat['IV_seg'] = ['Suspicious' if data_grpby_cat['IV'][i]>0.5
                                       else('High' if data_grpby_cat['IV'][i]<0.5 and data_grpby_cat['IV'][i]>=0.3 
                                          else('Medium' if data_grpby_cat['IV'][i]<0.3 and data_grpby_cat['IV'][i]>=0.1 
                                              else('Low' if data_grpby_cat['IV'][i]<0.1 and data_grpby_cat['IV'][i]>0.05 
                                                  else 'Not useful'))) for i in range(data_grpby_cat.shape[0])]

In [23]:
iv_df = pd.concat([data_grpby_cat, data_grpby_cont], axis = 0)
iv_df[iv_df['IV_seg'].isin(['Medium', 'Low', 'High'])].sort_values(by = 'IV', ascending = False)

,Variable,IV,IV_seg
67,lead_source_final,0.344369,High
41,Lead_source_channel,0.297172,Medium
1,B2B_FLAG,0.243670,Medium
5,CIR_CC_UTIL_PERC,0.238235,Medium
6,CIR_CC_UTIL_PERC_new,0.238235,Medium
117,unsec_enq_L12m_new,0.218149,Medium
43,TOT_enq_L12m_new,0.204786,Medium
119,utilization_percent_new,0.201699,Medium
118,utilization_percent,0.201563,Medium
114,round_utilization_percent,0.196044,Medium


In [24]:
# Dropping Unuseful IV columns

iv_vars =  list(iv_df.sort_values(by = 'IV', ascending = False)['Variable'])[:60] # taking top 60
iv_vars = [i for i in iv_vars if 'CIR_' not in i] # removing CIR columns as per discussion
print(len(iv_vars))
iv_df[iv_df['Variable'].isin(iv_vars)].sort_values(by = 'IV', ascending = False)

50


,Variable,IV,IV_seg
67,lead_source_final,0.344369,High
41,Lead_source_channel,0.297172,Medium
1,B2B_FLAG,0.243670,Medium
117,unsec_enq_L12m_new,0.218149,Medium
43,TOT_enq_L12m_new,0.204786,Medium
119,utilization_percent_new,0.201699,Medium
118,utilization_percent,0.201563,Medium
114,round_utilization_percent,0.196044,Medium
115,round_utilization_percent_new,0.193777,Medium
58,bureau_live_cc_bal_amt,0.164398,Medium


In [25]:
# Removing correlated variables from these variables

corr = data1[iv_vars].corr() > 0.7
col_list = list(corr.columns)
col_d = dict(enumerate(col_list))
corr_d = dict(zip(col_list, [[] for i in col_list]))

# Identifying correlated columns for each variable
for i in col_list:
    for j in range(0, len(col_list)):
        if corr[i][j] == True:
            if i != col_d[j]:
                corr_d[i].append(col_d[j])

# creating comparison of correlated columns using IV
x = iv_df[iv_df['Variable'].isin(col_list)][['Variable', 'IV']]
IV_d = dict(zip(list(x['Variable']), list(x['IV'])))
for i in corr_d:
    print("for col: ", i)
    print(i, np.round(IV_d[i], 3))
    if corr_d[i] != []:
        for j in corr_d[i]:
            pass
            print(j, np.round(IV_d[j], 3))
    print()

for col:  unsec_enq_L12m_new
unsec_enq_L12m_new 0.218
TOT_enq_L12m_new 0.205
CCPL_enq_L12m_new 0.154

for col:  TOT_enq_L12m_new
TOT_enq_L12m_new 0.205
unsec_enq_L12m_new 0.218
CCPL_enq_L12m_new 0.154

for col:  utilization_percent_new
utilization_percent_new 0.202
utilization_percent 0.202
round_utilization_percent 0.196
round_utilization_percent_new 0.194

for col:  utilization_percent
utilization_percent 0.202
utilization_percent_new 0.202
round_utilization_percent 0.196
round_utilization_percent_new 0.194

for col:  round_utilization_percent
round_utilization_percent 0.196
utilization_percent_new 0.202
utilization_percent 0.202
round_utilization_percent_new 0.194

for col:  round_utilization_percent_new
round_utilization_percent_new 0.194
utilization_percent_new 0.202
utilization_percent 0.202
round_utilization_percent 0.196

for col:  bureau_live_cc_bal_amt
bureau_live_cc_bal_amt 0.164
bureau_live_cc_sanc_amt 0.071
max_live_sanc_amt_new 0.061
max_cc_limit_new 0.058

for col:  CCPL

In [26]:
# Selecting correlated columns to remove

corr_cols = [
'TOT_enq_L12m_new',
'CCPL_enq_L12m_new',
'utilization_percent',
'round_utilization_percent',
'round_utilization_percent_new',
'bureau_live_cc_sanc_amt',
'max_live_sanc_amt_new',
'max_cc_limit_new',
'bureau_unsec_live_cnt',
'bureau_total_live_loan_cnt',
'bureau_Live_PL_amt',
'onus_latest_closed_cc_vintage',
'onus_latest_disb_loan_vintage',
'bureau_total_loan_cnt',
'offus_cc_limit',
'onus_sum_ECOM_all_loan_DISB_amt',
'CUST_CITY_TIER'
]

data_model = data1[iv_vars]
data_model.drop(corr_cols, axis = 1, inplace = True)
data_model.drop(['Lead_source_channel', 'B2B_Insta_Flag'], axis = 1, inplace = True)
print(len(data_model.columns))

31


In [27]:
woe_df.to_csv('woe_cc_df.csv')

In [28]:
#data_model['TOT_enq_L12m_new'] = data1['TOT_enq_L12m_new']
#data_model['round_utilization_percent_new'] = data1['round_utilization_percent_new']

from sklearn.preprocessing import MinMaxScaler
def scale_df(df):
    m = MinMaxScaler()
    for i in df._get_numeric_data().columns:
        df[i] = m.fit_transform(np.array(df[i]).reshape(-1, 1))
    return df

#data_model = scale_df(df = data_model.copy())

In [29]:
# Impute Categorical Columns with WOE

def impute_cat_woe(woe_df, data, cols_to_imp):
    band_woe = []
    cat_woe_df = woe_df[woe_df['Variable'].isin(cols_to_imp)]
    for var in cols_to_imp:
        n = cat_woe_df[cat_woe_df['Variable'] == var]
        for band in n.Band:
            woe = n[n['Band']==band].woe.values[0]
            band_woe.append((var, band, woe))
    for tup in band_woe:
        var, band, woe = tup
        data[var] = data[var].map(lambda x: woe if x == band else x)
    return data


num_cols = list(data_model._get_numeric_data().columns)
cat_cols = [c for c in data_model.columns if c not in num_cols]
data_model = impute_cat_woe(data_woe_cat_f, data = data_model.copy(), cols_to_imp = cat_cols)

In [30]:
predict = lambda prob, thresh: [int(i > thresh) for i in prob]

def predict_prob(data, model1, model2, wt1, wt2):
    p1 = np.array([i[1] for i in model1.predict_proba(data)])
    p2 = np.array([i[1] for i in model2.predict_proba(data)])
    p = wt1*p1 + wt2*p2
    return np.array(p)

def confusionMatrix(y, y_pred):
    print("Accuracy: ", accuracy_score(y, y_pred))
    print("F1_score: ", f1_score(y, y_pred))
    print("Precision Score: ", precision_score(y, y_pred))
    print("Recall Score: ", recall_score(y, y_pred))
    cm = confusion_matrix(y, y_pred, normalize = 'true')
    cmd = ConfusionMatrixDisplay(cm, display_labels=[0, 1])
    x = cmd.plot()

## GBM (32 variables)

In [23]:
x = data_model
y = data1['activation_flag_1MOB']
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, test_size = 0.30, random_state = 0)

gbc = GradientBoostingClassifier(n_estimators=500,learning_rate=0.05,random_state=100,max_features=5)
gbc.fit(x_train.values,y_train.values)
y_pred_GBM = gbc.predict(x_test.values)


#print("Train CM:")
#confusionMatrix(x_train, y_train, predict(x_train, gbc, 0.33))
#print()
#print("Test CM:")
#confusionMatrix(x_test, y_test, predict(x_test, gbc, 0.33))

In [24]:
# Obtain Train Deciles & KS

x_train_GBM = x_train.copy()
X_train_probs_GBM = gbc.predict_proba(x_train.values)
predictions_GBM =[i[1] for i in X_train_probs_GBM]
x_train_GBM['X_train_probs_GBM'] = predictions_GBM
x_train_GBM['activation_flag'] = y_train
ks_data_GBM = ks(data = x_train_GBM, target = 'activation_flag', prob = 'X_train_probs_GBM')
ks_data_GBM

KS is 34.5% at decile 5


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,0.563500,0.884842,5594,2851,20.50%,4.99%,20.50%,4.99%,15.5
2,0.481941,0.563495,4466,3979,16.37%,6.96%,36.87%,11.95%,24.9
3,0.419303,0.481941,3821,4624,14.00%,8.09%,50.87%,20.04%,30.8
4,0.362587,0.419303,3281,5164,12.02%,9.03%,62.90%,29.07%,33.8
5,0.309232,0.362563,2861,5583,10.49%,9.77%,73.38%,38.84%,34.5
6,0.258960,0.309227,2407,6038,8.82%,10.56%,82.21%,49.40%,32.8
7,0.206020,0.258953,1892,6553,6.93%,11.46%,89.14%,60.86%,28.3
8,0.159166,0.206019,1517,6928,5.56%,12.12%,94.70%,72.98%,21.7
9,0.100160,0.159158,1026,7419,3.76%,12.98%,98.46%,85.96%,12.5


In [25]:
# Obtain test deciles & KS

ks_test = pd.DataFrame()
ks_test['Probs'] = [i[1] for i in gbc.predict_proba(x_test)]
ks_test['activation_flag'] = y_test.values
prob = lambda k: round(ks_data_GBM['min_prob'].iloc[k],5)

ks_test['decile'] = [1 if ks_test['Probs'][i]>= prob(0)
else(2 if ks_test['Probs'][i]>= prob(1)
else(3 if ks_test['Probs'][i]>= prob(2)
else(4 if ks_test['Probs'][i]>= prob(3)
else(5 if ks_test['Probs'][i]>= prob(4)
else(6 if ks_test['Probs'][i]>= prob(5)
else(7 if ks_test['Probs'][i]>= prob(6)
else(8 if ks_test['Probs'][i]>= prob(7)
else(9 if ks_test['Probs'][i]>= prob(8)
else(10))))))))) for i in range(ks_test.shape[0])]

ks_test_df = ks_test.groupby('decile').count().reset_index()[['decile','Probs']]
ks_test_df.rename({'Probs': 'Population'}, axis = 1, inplace = True)
ks_test_df['Activation'] = list(ks_test.groupby('decile').sum()['activation_flag'])

event_rate = (ks_test_df['Activation'] / sum(y_test))
nonevent_rate = ((ks_test_df['Population'] - ks_test_df['Activation']) / (len(y_test) - sum(y_test)))
cum_eventrate=(ks_test_df['Activation'] / sum(y_test)).cumsum()
cum_noneventrate=((ks_test_df['Population'] - ks_test_df['Activation']) / (len(y_test) - sum(y_test))).cumsum()
ks_test_df['KS'] = np.round(cum_eventrate - cum_noneventrate, 3) * 100
ks_test_df

,decile,Population,Activation,KS
0,1,3624,2285,14.1
1,2,3605,1868,23.0
2,3,3600,1619,28.7
3,4,3688,1423,31.6
4,5,3556,1183,32.1
5,6,3583,1068,30.9
6,7,3697,894,27.1
7,8,3655,708,21.2
8,9,3606,460,12.3
9,10,3579,186,0.0


In [26]:
# select top columns according to feature importance

curr_model_cols = list(x.columns)
d1 = dict(zip(curr_model_cols, gbc.feature_importances_))
d2 = sorted(d1, key = lambda k: d1[k], reverse = True)
feat_imp = dict(zip(d2, [d1[i] for i in d2]))
l = [i for i in list(feat_imp) if i not in ('B2B_FLAG', 'TAG')]
l

['lead_source_final',
 'bureau_live_cc_bal_amt',
 'utilization_percent_new',
 'unsec_enq_L12m_new',
 'TOT_LIVE_UNSEC_LOAN1',
 'OFFER_VINTAGE',
 'BEH_DAYS_GAP_BETWEEN_2_PURCHASE',
 'onus_offus_cc_limit_ratio',
 'bureau_Last_Loan_vintage',
 'bureau_live_pl_cnt',
 'SMS_click_last_12M_flag1',
 'App_installation_flag1',
 'onus_latest_B2B_loan_vintage',
 'City_Tier_NOV21',
 'onus_first_disb_loan_vintage',
 'TOT_UNSEC_LOAN_new',
 'age_nov21',
 'BEH_DIGITAL_SAVVY',
 'BEH_BFL_LOYALTY_ATTRITION',
 'onus_Sum_live_loan_DISB_amt',
 'onus_avg_live_loan_DISB_amt',
 'onus_ECOM_loan_cnt_ever',
 'ECOM_PROPENSITY_TO_BUY',
 'BEH_SALES_FINANCE_PURCHASE_BEHAVIOR',
 'BEH_PURCHASE_BEHAVIOR_ON_EMI_CARD',
 'ECOM_COUNT_EVER_BOUGHT',
 'ECOM_FLIPKART_MOBILE_BOUGHT',
 'CUST_WALLET_DIGITIZE_FLAG',
 'ECOM_EVER_BOUGHT']

## GBM with Variable Reduction

In [31]:
# Final model columns chosen

model_cols = [
'lead_source_final',
'unsec_enq_L12m_new', #
'utilization_percent_new',
'TOT_LIVE_UNSEC_LOAN1',
'onus_offus_cc_limit_ratio',
'bureau_Last_Loan_vintage',
'SMS_click_last_12M_flag1',
'App_installation_flag1',
'onus_latest_B2B_loan_vintage',
'age_nov21',
'onus_ECOM_loan_cnt_ever',
'onus_Sum_live_loan_DISB_amt',
'OFFER_VINTAGE',
'bureau_live_pl_cnt'
]

x1 = data_model[model_cols]
y = data1['activation_flag_1MOB']

x_train, x_test, y_train, y_test = train_test_split(x1, y, stratify = y, test_size = 0.25, random_state = 0)

In [32]:
# Fine Tune XGB Classifier
from xgboost import XGBClassifier
import optuna

def objective(trial):
    train_x, test_x, train_y, test_y = train_test_split(x1, y, test_size=0.25)
    param = {
        "objective": "binary:logistic",
        "eval_metric": ["auc", "map", "aucpr"],
        "booster": "gbtree",
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 2.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 2.0),
    }
    if param["booster"] == "gbtree":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
        param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
        param['colsample_bytree']= trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0])
        param['subsample']= trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0])
        param['learning_rate']= trial.suggest_categorical('learning_rate', [0.01,0.015,0.02,0.025, 0.03])
        param['n_estimators']= trial.suggest_int('n_estimators',100, 1000)
        param['min_child_weight']= trial.suggest_int('min_child_weight', 1, 300)
        param['scale_pos_weight'] = trial.suggest_loguniform("scale_pos_weight", 1.5, 3.5)
        

    xgb = XGBClassifier(**param)
    bst = xgb.fit(train_x, train_y, eval_set=[(test_x, test_y)])
    preds = bst.predict(test_x)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(test_y, pred_labels)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=64)

[I 2022-08-04 16:30:12,573] A new study created in memory with name: no-name-7147b5f7-1210-4970-92e2-2ae1c7421bb1


[0]	validation_0-auc:0.70139	validation_0-map:0.50669	validation_0-aucpr:0.50679
[1]	validation_0-auc:0.70667	validation_0-map:0.51471	validation_0-aucpr:0.51500
[2]	validation_0-auc:0.70751	validation_0-map:0.51658	validation_0-aucpr:0.51684
[3]	validation_0-auc:0.70823	validation_0-map:0.51765	validation_0-aucpr:0.51766
[4]	validation_0-auc:0.70861	validation_0-map:0.51811	validation_0-aucpr:0.51812
[5]	validation_0-auc:0.71000	validation_0-map:0.52050	validation_0-aucpr:0.52043
[6]	validation_0-auc:0.71007	validation_0-map:0.52034	validation_0-aucpr:0.52028
[7]	validation_0-auc:0.71034	validation_0-map:0.52034	validation_0-aucpr:0.52028
[8]	validation_0-auc:0.71096	validation_0-map:0.52154	validation_0-aucpr:0.52151
[9]	validation_0-auc:0.71096	validation_0-map:0.52120	validation_0-aucpr:0.52117
[10]	validation_0-auc:0.71125	validation_0-map:0.52166	validation_0-aucpr:0.52161
[11]	validation_0-auc:0.71128	validation_0-map:0.52159	validation_0-aucpr:0.52152
[12]	validation_0-auc:0.71

KeyboardInterrupt: 

In [25]:
study.best_params

{'lambda': 4.6331708500002277e-07,
 'alpha': 2.4586107957170375e-06,
 'max_depth': 9,
 'eta': 2.8270044288178234e-07,
 'gamma': 0.0007812593431028508,
 'grow_policy': 'lossguide',
 'colsample_bytree': 0.6,
 'subsample': 0.6,
 'learning_rate': 0.02,
 'n_estimators': 192,
 'min_child_weight': 246,
 'scale_pos_weight': 1.5058549944250361}

In [33]:
params = {'lambda': 4.6331708500002277e-07,
 'alpha': 2.4586107957170375e-06,
 'max_depth': 9,
 'eta': 2.8270044288178234e-07,
 'gamma': 0.0007812593431028508,
 'grow_policy': 'lossguide',
 'colsample_bytree': 0.6,
 'subsample': 0.6,
 'learning_rate': 0.02,
 'n_estimators': 192,
 'min_child_weight': 246,
 'scale_pos_weight': 1.5058549944250361}
gbc = XGBClassifier(**params)

gbc.fit(x_train,y_train)
#confusionMatrix(y = y_test, y_pred = predict([i[1] for i in gbc.predict_proba(x_test)], 0.5))

XGBClassifier(alpha=2.4586107957170375e-06, base_score=0.5, booster='gbtree',
              callbacks=None, colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eta=2.8270044288178234e-07,
              eval_metric=None, gamma=0.0007812593431028508, gpu_id=-1,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints='', lambda=4.6331708500002277e-07,
              learning_rate=0.02, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=246,
              missing=nan, monotone_constraints='()', n_estimators=192,
              n_jobs=0, num_parallel_tree=1, predictor='auto', ...)

In [37]:
ks_data_GBM.to_csv('ks_cc.csv')

In [34]:
x_train_GBM = x_train.copy()
X_train_probs_GBM = gbc.predict_proba(x_train)
predictions_GBM = [i[1] for i in X_train_probs_GBM] 
x_train_GBM['X_train_probs_GBM'] = predictions_GBM
x_train_GBM['activation_flag'] = y_train
ks_data_GBM = ks(data = x_train_GBM, target = 'activation_flag', prob = 'X_train_probs_GBM')
ks_data_GBM

KS is 33.2% at decile 5


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,0.638690,0.833235,5771,3277,19.74%,5.35%,19.74%,5.35%,14.4
2,0.570407,0.638680,4785,4262,16.37%,6.96%,36.11%,12.31%,23.8
3,0.513296,0.570404,4047,5002,13.84%,8.17%,49.95%,20.48%,29.5
4,0.459763,0.513296,3555,5493,12.16%,8.97%,62.11%,29.44%,32.7
5,0.406707,0.459761,3032,6016,10.37%,9.82%,72.48%,39.27%,33.2
6,0.354895,0.406703,2663,6385,9.11%,10.43%,81.59%,49.69%,31.9
7,0.298310,0.354882,2117,6931,7.24%,11.32%,88.83%,61.01%,27.8
8,0.241558,0.298306,1654,7394,5.66%,12.07%,94.49%,73.08%,21.4
9,0.171733,0.241551,1125,7923,3.85%,12.94%,98.34%,86.02%,12.3


In [35]:
ks_test = pd.DataFrame()
ks_test['Probs'] = [i[1] for i in gbc.predict_proba(x_test.values)]
ks_test['activation_flag'] = y_test.values
prob = lambda k: round(ks_data_GBM['min_prob'].iloc[k],5)

ks_test['decile'] = [1 if ks_test['Probs'][i]>= prob(0)
else(2 if ks_test['Probs'][i]>= prob(1)
else(3 if ks_test['Probs'][i]>= prob(2)
else(4 if ks_test['Probs'][i]>= prob(3)
else(5 if ks_test['Probs'][i]>= prob(4)
else(6 if ks_test['Probs'][i]>= prob(5)
else(7 if ks_test['Probs'][i]>= prob(6)
else(8 if ks_test['Probs'][i]>= prob(7)
else(9 if ks_test['Probs'][i]>= prob(8)
else(10))))))))) for i in range(ks_test.shape[0])]

ks_test_df = ks_test.groupby('decile').count().reset_index()[['decile','Probs']]
ks_test_df.rename({'Probs': 'Population'}, axis = 1, inplace = True)
ks_test_df['Activation'] = list(ks_test.groupby('decile').sum()['activation_flag'])

event_rate = (ks_test_df['Activation'] / sum(y_test))
nonevent_rate = ((ks_test_df['Population'] - ks_test_df['Activation']) / (len(y_test) - sum(y_test)))
cum_eventrate=(ks_test_df['Activation'] / sum(y_test)).cumsum()
cum_noneventrate=((ks_test_df['Population'] - ks_test_df['Activation']) / (len(y_test) - sum(y_test))).cumsum()
ks_test_df['KS'] = np.round(cum_eventrate - cum_noneventrate, 3) * 100
ks_test_df

,decile,Population,Activation,KS
0,1,2960,1833,13.3
1,2,3091,1605,22.5
2,3,2933,1287,27.6
3,4,3000,1167,30.6
4,5,2924,1002,31.5
5,6,3034,920,30.6
6,7,3055,755,27.1
7,8,3120,608,21.0
8,9,3052,396,12.0
9,10,2992,172,-0.0


### Scoring Jan '22 STR

In [44]:
df_score = pd.read_csv('etb_cap_cc_score.csv', header = None)
df_score.replace('NULL', np.nan, inplace = True)
df_score.head()

cols = [
'cust_id__c',
'apprefno',
'lead_source_final',
'bureau_Last_Loan_vintage',
'age_nov21',
'App_installation_flag1',
'TOT_LIVE_UNSEC_LOAN1',
'bureau_live_pl_cnt',
'unsec_enq_L12m_new',
'utilization_percent_new',
'bureau_live_cc_bal_amt',
'SMS_click_last_12M_flag1',
'OFFER_VINTAGE',
'onus_offus_cc_limit_ratio',
'bureau_unsec_live_cnt',
'onus_latest_B2B_loan_vintage',
'CCPL_enq_L12m_new',
'BUREAUCUSTOMERID',
'carded_non_carded',
'activation_flag_1MOB',
'onus_ECOM_loan_cnt_ever',
'onus_Sum_live_loan_DISB_amt',
'sec_enq_L12m_new',
'num_ECF_REMI_1yr'
]

df_score.rename(dict(enumerate(cols)), inplace = True, axis = 1)
df_score = df_score[df_score['cust_id__c'].notnull()]

In [45]:
model_cols.append('activation_flag_1MOB')
df_score = df_score[model_cols]
model_cols.remove('activation_flag_1MOB')

for i in df_score.columns:
    print(i, df_score[i].isna().sum())

lead_source_final 0
unsec_enq_L12m_new 13442
utilization_percent_new 5539
TOT_LIVE_UNSEC_LOAN1 2741
onus_offus_cc_limit_ratio 144
bureau_Last_Loan_vintage 2741
SMS_click_last_12M_flag1 0
App_installation_flag1 0
onus_latest_B2B_loan_vintage 7139
age_nov21 5081
onus_ECOM_loan_cnt_ever 4497
onus_Sum_live_loan_DISB_amt 4497
OFFER_VINTAGE 0
bureau_live_pl_cnt 2741
activation_flag_1MOB 0


In [46]:
# Missing value Treatment

#df_score['unsec_enq_L12m_new'].fillna(-99999, inplace = True)
df_score['utilization_percent_new'].fillna(0, inplace = True)
#df_score['bureau_live_cc_bal_amt'].fillna(df_score['bureau_live_cc_bal_amt'].mean(), inplace = True)

df_score['bureau_live_pl_cnt'].fillna(0, inplace = True)

#df_score['bureau_Last_Loan_vintage'].fillna(df_score['bureau_Last_Loan_vintage'].mean(), inplace = True)
df_score['onus_offus_cc_limit_ratio'].fillna(0, inplace = True)
df_score['onus_latest_B2B_loan_vintage'].fillna(df_score['onus_latest_B2B_loan_vintage'].mean(), inplace = True)
df_score['TOT_LIVE_UNSEC_LOAN1'].fillna(0, inplace = True)

df_score['onus_Sum_live_loan_DISB_amt'].fillna(0, inplace = True)
df_score['age_nov21'].fillna(df_score['age_nov21'].mean(), inplace = True)

df_score['onus_ECOM_loan_cnt_ever'].fillna(-99999, inplace = True)


In [47]:
# Oulier Treatment
df_cc = df_score.drop('activation_flag_1MOB', axis = 1)
num_col = df_cc._get_numeric_data().columns
df_cc[num_col] = df_cc[num_col].apply(lambda x: x.clip(*x.quantile([0.05, 0.99])))

# Categorical Imputation
num_cols = list(df_cc._get_numeric_data().columns)
cat_cols = [c for c in df_cc.columns if c not in num_cols]

data_model1 = impute_cat_woe(data_woe_cat_f, data = df_cc.copy(), cols_to_imp = cat_cols)

In [48]:
# Obtaining KS

X = data_model1[model_cols]
y = df_score['activation_flag_1MOB']

ks_score = pd.DataFrame()
ks_score['Probs'] = [i[1] for i in gbc.predict_proba(X)]
ks_score['activation_flag'] = y.values
prob = lambda k: round(ks_data_GBM['min_prob'].iloc[k],5)

ks_score['decile'] = [1 if ks_score['Probs'][i]>= prob(0)
else(2 if ks_score['Probs'][i]>= prob(1)
else(3 if ks_score['Probs'][i]>= prob(2)
else(4 if ks_score['Probs'][i]>= prob(3)
else(5 if ks_score['Probs'][i]>= prob(4)
else(6 if ks_score['Probs'][i]>= prob(5)
else(7 if ks_score['Probs'][i]>= prob(6)
else(8 if ks_score['Probs'][i]>= prob(7)
else(9 if ks_score['Probs'][i]>= prob(8)
else(10))))))))) for i in range(ks_score.shape[0])]

ks_score_df = ks_score.groupby('decile').count().reset_index()[['decile','Probs']]
ks_score_df.rename({'Probs': 'Population'}, axis = 1, inplace = True)
ks_score_df['Activation'] = list(ks_score.groupby('decile').sum()['activation_flag'])

event_rate = (ks_score_df['Activation'] / sum(y))
nonevent_rate = ((ks_score_df['Population'] - ks_score_df['Activation']) / (len(y) - sum(y)))
cum_eventrate=(ks_score_df['Activation'] / sum(y)).cumsum()
cum_noneventrate=((ks_score_df['Population'] - ks_score_df['Activation']) / (len(y) - sum(y))).cumsum()
ks_score_df['KS'] = np.round(cum_eventrate - cum_noneventrate, 3) * 100
ks_score_df

,decile,Population,Activation,KS
0,1,4153,2588,10.4
1,2,4526,2400,18.0
2,3,4635,2284,24.3
3,4,4810,2069,28.1
4,5,4901,1929,30.4
5,6,5093,1602,29.3
6,7,5136,1517,27.2
7,8,5117,1247,22.9
8,9,5645,851,13.4
9,10,6350,642,-0.0


In [49]:
# Population distribution
ks_score_df['Population']*100/ks_score_df['Population'].sum()

0     8.245642
1     8.986221
2     9.202637
3     9.550093
4     9.730771
5    10.111980
6    10.197355
7    10.159631
8    11.207958
9    12.607712
Name: Population, dtype: float64

In [ ]:
# Save model as pickle file
pkl_filename = "xgb_cc.pkl" 
with open(pkl_filename, 'wb') as file:
    pickle.dump(gbc, file)

In [51]:
import joblib
joblib.dump(gbc, 'xgb_cc_model_final.pkl')

['xgb_cc_model_final.pkl']

In [52]:
x = joblib.load('xgb_cc_model_final.pkl')

In [54]:
x.predict_proba(data_model[model_cols])

array([[0.6423979 , 0.35760215],
       [0.4094454 , 0.5905546 ],
       [0.689051  , 0.31094903],
       ...,
       [0.4346059 , 0.5653941 ],
       [0.6684005 , 0.33159944],
       [0.6359736 , 0.3640264 ]], dtype=float32)

In [104]:
### Computing Characteristic Stability Index -- because population instability in 9th decile

def CSI(model_cols, train_data, score_data, target_var):
    csi_list = []
    num_cols = list(train_data[model_cols]._get_numeric_data().columns)
    cat_cols = [c for c in model_cols if c not in num_cols]

    for col in model_cols:
        # categorical columns
        if col in cat_cols:
            csi_tr = train_data.groupby(col).count()[target_var].reset_index()
            csi_tr[target_var] = csi_tr[target_var]/csi_tr[target_var].sum()
            csi_te = score_data.groupby(col).count()[target_var].reset_index()
            csi_te[target_var] = csi_te[target_var]/csi_te[target_var].sum()
            pop_diff = np.array(csi_tr[target_var] - csi_te[target_var])
            logdiff = np.log(csi_tr[target_var]/csi_te[target_var])
            csi = sum(pop_diff * logdiff)
            csi_list.append(csi)
        # numerical columns
        if col in num_cols:
            csi_tr = pd.qcut(train_data[col], q = 10, duplicates = "drop")
            csi_tr1 = csi_tr.reset_index()
            v1 = np.array(csi_tr1.groupby(col).count()['index'])
            v1 = v1/sum(v1)
            pop = []
            for i in csi_tr.unique():
                pop.append(sum(score_data[col].map(lambda x: 1 if x in i else 0)))
            v2 = np.array(pop)/sum(pop)
            pop_diff = np.array(v1 - v2)
            logdiff = np.log(v1/v2)
            csi = sum(pop_diff * logdiff)
            csi_list.append(csi)
    return dict(zip(model_cols, csi_list))

df_cc['activation_flag_1MOB'] = df_score['activation_flag_1MOB']
CSI(model_cols, data1, df_cc, 'activation_flag_1MOB')

{'lead_source_final': 0.012021311430899562,
 'utilization_percent_new': 0.5369193926234731,
 'TOT_LIVE_UNSEC_LOAN1': 0.12445072293511754,
 'onus_offus_cc_limit_ratio': 0.02576740495817486,
 'bureau_Last_Loan_vintage': 0.5451309250591159,
 'SMS_click_last_12M_flag1': 0.01337889292247218,
 'App_installation_flag1': 0.0011435512350202266,
 'onus_latest_B2B_loan_vintage': 0.3044977400925854,
 'age_nov21': 0.19887234140454096,
 'onus_ECOM_loan_cnt_ever': 0.0031473046029352296,
 'onus_Sum_live_loan_DISB_amt': 0.20919718013564548,
 'OFFER_VINTAGE': 0.7116441603707797}

In [55]:
ks_test_df.to_csv('ks_test.csv')
ks_data_GBM.to_csv('ks_train.csv')
ks_score_df.to_csv('ks_score.csv')

In [56]:
for i in model_cols:
    print(i)

lead_source_final
unsec_enq_L12m_new
utilization_percent_new
TOT_LIVE_UNSEC_LOAN1
onus_offus_cc_limit_ratio
bureau_Last_Loan_vintage
SMS_click_last_12M_flag1
App_installation_flag1
onus_latest_B2B_loan_vintage
age_nov21
onus_ECOM_loan_cnt_ever
onus_Sum_live_loan_DISB_amt
OFFER_VINTAGE
bureau_live_pl_cnt


In [58]:
lx = set(model_cols).union(set([
'lead_source_final',
'unsec_enq_L12m_new',
'age_nov21',
'OFFER_VINTAGE',
'CCPL_enq_L12m_new',
'onus_latest_B2B_loan_vintage',
'App_installation_flag1',
'SMS_click_last_12M_flag1',
'bureau_Last_Loan_vintage',
'bureau_live_pl_cnt',
'onus_ECOM_loan_cnt_ever'
]))

for i in lx:
    print(i)

onus_ECOM_loan_cnt_ever
unsec_enq_L12m_new
SMS_click_last_12M_flag1
onus_Sum_live_loan_DISB_amt
age_nov21
onus_offus_cc_limit_ratio
bureau_Last_Loan_vintage
CCPL_enq_L12m_new
lead_source_final
utilization_percent_new
TOT_LIVE_UNSEC_LOAN1
bureau_live_pl_cnt
OFFER_VINTAGE
onus_latest_B2B_loan_vintage
App_installation_flag1
